In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=a1dad408dcb1360d45ebe78a12e6384b841d5e80001097ecca8656dbdd057f4b
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [0]:
## 諸々の import

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
import cv2
# from tqdm.auto import tqdm
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
# from torchsummary import summary
import gc


dataset_dir = '../dataset'
# dataset_dir = '/kaggle/input/bengaliai-cv19'
train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')

model_dir = '../trained_models'
# model_dir = '/kaggle/input/models'

In [0]:
from preprocess import *
from model.CNN import model
# from model.efficientnet import model
# from model.resnet18 import model

model = model()
model = try_gpu(model)

criterion1 = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

In [0]:
## Parameters

# resize後のサイズ
HEIGHT = 64
WIDTH = 64

# 画像を3次元にするかどうか（EfficientNetなどを使うときはTrue）
enable_3d = True

# True なら Cross Validation を実施する
# Kaggle に提出するときは False にしてください
do_validation = True

# True なら submission.csv を生成する
create_submission = False

val_perc = 0.2  # validation set の割合（クロスバリデーション）
epochs = 4

model_dict_path = '/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/model/cnn_dict'


In [0]:
## train関数、test関数

def train(model, epoch, train_loader):
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()

        
def test(model, test_loader):
    model.eval()

    size = len(test_loader.dataset)
    pred_r, pred_v, pred_c = np.zeros(size), np.zeros(size), np.zeros(size)
    true_r, true_v, true_c = np.zeros(size), np.zeros(size), np.zeros(size)
    index = 0
    
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs) 
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        for i in range(inputs.size(0)):
            pred_r[index] = root_pred[i]
            pred_v[index] = vowel_pred[i]
            pred_c[index] = consonant_pred[i]
            true_r[index] = root_l[i]
            true_v[index] = vowel_l[i]
            true_c[index] = consonant_l[i]
            index += 1

    recall_r = recall_score(true_r, pred_r, average='macro')
    recall_v = recall_score(true_v, pred_v, average='macro')
    recall_c = recall_score(true_c, pred_c, average='macro')
    final_score = (2.*recall_r + recall_v + recall_c) / 4.

    print(f'Root Recall: {recall_r:.5f}')
    print(f'Vowel Recall: {recall_v:.5f}')
    print(f'Consonant Recall: {recall_c:.5f}')
    print(f'Score: {final_score:.5f}')


In [14]:
## データの読み込み

X_all = np.empty((0, HEIGHT*WIDTH))
Y_root_all = np.empty((0, 168))
Y_vowel_all = np.empty((0, 11))
Y_cons_all = np.empty((0, 7))

for parq_i in range(4):
    print(f'Parquet {parq_i} を読み込み中')
    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)
    
    X = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])
    X_resized = resize(X, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8) # astype(np.uint8)をしてあげることで後で cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB) が実行できるようになる
    
    Y_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    X_all = np.append(X_all, X_resized, axis=0)
    Y_root_all = np.append(Y_root_all, Y_root, axis=0)
    Y_vowel_all = np.append(Y_vowel_all, Y_vowel, axis=0)
    Y_cons_all = np.append(Y_cons_all, Y_cons, axis=0)

    del X
    del X_resized
    del Y_root
    del Y_vowel 
    del Y_cons 
    gc.collect()

print(X_all.shape)
print(Y_root_all.shape)
print(Y_vowel_all.shape)
print(Y_cons_all.shape)

Y_all = [Y_root_all, Y_vowel_all, Y_cons_all]

trainval_dataset = MyDataset(X_all, Y_all, enable_3d=enable_3d, H=HEIGHT, W=WIDTH)

del X_all
del Y_root_all
del Y_vowel_all
del Y_cons_all
del Y_all
gc.collect()

Parquet 0 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 1 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 2 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 3 を読み込み中
Resizing raw image... / 前処理実行中…
(200840, 4096)
(200840, 168)
(200840, 11)
(200840, 7)


0

In [15]:
## 訓練ループ / Training Loop

print('==========================')
print('Starting training.')

if do_validation:
    n_samples = len(trainval_dataset)
    train_size = int(len(trainval_dataset)*(1.0 - val_perc))
    val_size = n_samples - train_size
    print(f'train size: {train_size}, validation size: {val_size}')

    subset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

    if enable_3d:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        ))

    else:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        ))

    del trainval_dataset
    del subset
    gc.collect()

    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=0)

    for epoch in range(1,epochs+1):
        print(f'Epoch number {epoch}')
        train(model, epoch, train_loader)
        test(model, val_loader)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_dict_path)

    # メモリ節約
    del train_dataset
    del val_dataset
    del train_loader
    del val_loader
    gc.collect()

else:
    n_samples = len(trainval_dataset)
    print(f'train size: {n_samples}')

    if enable_3d:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        )
    
    else:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        )

    train_loader = DataLoader(dataset=trainval_dataset, batch_size=32, shuffle=True, num_workers=4)

    for epoch in range(1,epochs+1):
        print(f'Epoch number {epoch}')
        train(model, epoch, train_loader)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_dict_path)

    # メモリ節約
    del trainval_dataset
    del train_loader
    gc.collect()


Starting training.
train size: 160672, validation size: 40168
Epoch number 1


RuntimeError: ignored